Define the Widgets for variables that need to be parameterized in this notebook

In [0]:
CREATE WIDGET TEXT secure_core_catalog_name DEFAULT "";
CREATE WIDGET TEXT landing_to_vault_load_tracking_table DEFAULT "";
CREATE WIDGET TEXT vault_to_anonymized_load_tracking_table DEFAULT "";
CREATE WIDGET TEXT vault_to_individual_attribute_load_tracking_table DEFAULT "";
CREATE WIDGET TEXT secure_core_group_name DEFAULT "";
CREATE WIDGET TEXT secure_core_pci_pii_attribute_table_name DEFAULT "";

Create the tracking table for maintaining the last loaded version for each of the tables in the landing catalog. The last loaded version will be used to determine if there is a new version of data ingested into the landing tables

In [0]:
CREATE TABLE IF NOT EXISTS `${secure_core_catalog_name}`.default.${landing_to_vault_load_tracking_table} (
  catalog_name STRING,
  schema_name STRING,
  table_name STRING,
  last_loaded_version INT,
  datetime TIMESTAMP
)

Create the tracking table for maintaining the last loaded version for each of the tables in the vault catalog. The last loaded version will be used to determine if there is a new version of data ingested into the vault tables

In [0]:
CREATE TABLE IF NOT EXISTS `${secure_core_catalog_name}`.default.${vault_to_anonymized_load_tracking_table} (
  catalog_name STRING,
  schema_name STRING,
  table_name STRING,
  last_loaded_version INT,
  datetime TIMESTAMP
)

In [0]:
CREATE TABLE IF NOT EXISTS `${secure_core_catalog_name}`.default.${vault_to_individual_attribute_load_tracking_table} (
  catalog_name STRING,
  schema_name STRING,
  table_name STRING,
  last_loaded_version INT,
  datetime TIMESTAMP
)

Create the table to maintain the PCI and PII attributes and what are its corresponding KEK (Key Encryption Key) and DSK (Data Salt Key) names

In [0]:
CREATE TABLE IF NOT EXISTS `${secure_core_catalog_name}`.default.${secure_core_pci_pii_attribute_table_name} (
  pci_pii_attribute_name STRING,
  kek_name STRING,
  dsk_name STRING
)

Insert the PCI and PII attributes and it's corresponding KEK (Key Encryption Key) and DSK (Data Salt Key) names

In [0]:
INSERT INTO `${secure_core_catalog_name}`.default.${secure_core_pci_pii_attribute_table_name}
SELECT 'name', 'secure-core-key-encryption-key-name', 'secure-core-data-salt-key-name' union
SELECT 'email', 'secure-core-key-encryption-key-email', 'secure-core-data-salt-key-email' union
SELECT 'phone', 'secure-core-key-encryption-key-phone', 'secure-core-data-salt-key-phone' union
SELECT 'dob', 'secure-core-key-encryption-key-dob', 'secure-core-data-salt-key-dob' union
SELECT 'ssn', 'secure-core-key-encryption-key-ssn', 'secure-core-data-salt-key-ssn' union
SELECT 'pan', 'secure-core-key-encryption-key-pan', 'secure-core-data-salt-key-pan' union
SELECT 'address', 'secure-core-key-encryption-key-address', 'secure-core-data-salt-key-address' union
SELECT 'exp-date', 'secure-core-key-encryption-key-exp-date', 'secure-core-data-salt-key-exp-date'

Masking function for PAN. Show only the last 4 digits for users who do not have privilege to see the clear text values

In [0]:
/*Masking function for PAN. Show only the last 4 digits for users who do not have privilege to see the clear text values*/
CREATE OR REPLACE FUNCTION `${secure_core_catalog_name}`.default.pan_mask(pan STRING)
  RETURN CASE WHEN is_member('${secure_core_group_name}') THEN pan
    ELSE
      CONCAT(
        REPEAT('X', LENGTH(pan) - 4), --middle digits
        RIGHT(pan, 4) --last 4 digits
      ) 
  END;

Masking function for Exp Date. Show only the year for users who do not have privilege to see the clear text values

In [0]:
/*Masking function for Exp Date. Show only the year for users who do not have privilege to see the clear text values*/
CREATE OR REPLACE FUNCTION `${secure_core_catalog_name}`.default.exp_date_mask(exp_date STRING)
  RETURN CASE WHEN is_member('${secure_core_group_name}') THEN exp_date
    ELSE
      CONCAT('20', RIGHT(exp_date, 2))
  END;

Masking function for Name. Show the first the first name and the first character of the last name for users who do not have privilege to see the clear text values

In [0]:
/*Masking function for Name. Show the first the first name and the first character of the last name for users who do not have privilege to see the clear text values*/
CREATE OR REPLACE FUNCTION `${secure_core_catalog_name}`.default.name_mask(name STRING)
  RETURN CASE WHEN is_member('${secure_core_group_name}') THEN name
    ELSE
      CONCAT(SPLIT(name, ' ')[0], ' ', LEFT(SPLIT(name, ' ')[1],1), '.')
    END;

Masking function for Address. Show the first few character of the address for users who do not have privilege to see the clear text values

In [0]:
/*Masking function for Address. Show the first few character of the address for users who do not have privilege to see the clear text values*/
CREATE OR REPLACE FUNCTION `${secure_core_catalog_name}`.default.address_mask(address STRING)
  RETURN CASE WHEN is_member('${secure_core_group_name}') THEN address
    ELSE
      CONCAT(LEFT(address, 10), '...')
    END;

Masking function for Email. Show the first character of the email and the rest as **** followed @domain.com for users who do not have privilege to see the clear text values

In [0]:
/*Masking function for Email. Show the first character of the email and the rest as **** followed @domain.com for users who do not have privilege to see the clear text values*/
CREATE OR REPLACE FUNCTION `${secure_core_catalog_name}`.default.email_mask(email STRING)
  RETURN CASE WHEN is_member('${secure_core_group_name}') THEN email
    ELSE
      CONCAT(LEFT(email, 1), '*****', SUBSTRING(email, INSTR(email, '@'), LENGTH(email)))
    END;

Masking function for Phone Number. Show only the last 4 digits of the phone and the rest as **** for users who do not have privilege to see the clear text values

In [0]:
/*Masking function for Phone Number. Show only the last 4 digits of the phone and the rest as **** for users who do not have privilege to see the clear text values*/
CREATE OR REPLACE FUNCTION `${secure_core_catalog_name}`.default.phone_mask(phone STRING)
  RETURN CASE WHEN is_member('${secure_core_group_name}') THEN phone
    ELSE
      CONCAT(REPEAT('*', LENGTH(phone) -4), RIGHT(phone, 4))
    END;

Masking function for Data of Birth. Show only the year of the Data of Birth for users who do not have privilege to see the clear text values

In [0]:
/*Masking function for Data of Birth. Show only the year of the Data of Birth for users who do not have privilege to see the clear text values*/
CREATE OR REPLACE FUNCTION `${secure_core_catalog_name}`.default.dob_mask(dob STRING)
  RETURN CASE WHEN is_member('${secure_core_group_name}') THEN dob
    ELSE
      CAST(YEAR(dob) AS STRING)
    END;

Masking function for SSN. Show only the last 4 digits of SSN for users who do not have privilege to see the clear text values

In [0]:
/*Masking function for SSN. Show only the last 4 digits of SSN for users who do not have privilege to see the clear text values*/
CREATE OR REPLACE FUNCTION `${secure_core_catalog_name}`.default.ssn_mask(ssn STRING)
  RETURN CASE WHEN is_member('${secure_core_group_name}') THEN ssn
    ELSE
      CONCAT('***-**-', RIGHT(ssn, 4))
    END;